In [ ]:
import os
import json
import re
from dotenv import load_dotenv
from collections import defaultdict

from get_data_from_or import get_client
import openreview
from datetime import datetime


In [ ]:
load_dotenv("credentials.env")
client = get_client(os.getenv("OR_API_VERSION"))

In [ ]:
notes = client.get_notes(invitation='ICLR.cc/2019/Conference/-/Blind_Submission', details="directReplies")

In [ ]:
print(len(notes))
for note in notes:
    for reply in note.details["directReplies"]:
        if reply["invitation"].endswith("Official_Review"):
            for signature in reply["signatures"]:
                if signature.startswith("~"):
                    print(json.dumps(reply, indent=3))
                    break

In [ ]:
venues = client.get_group(id='venues').members

In [ ]:
print(len(venues))

Single Blind submission which have peer reviews

In [ ]:
data = {}
for venue in venues:
    data[venue] = client.get_all_notes(invitation=f"{venue}/-/Submission", details="directReplies")

In [ ]:
peer_reviews = defaultdict(dict)
for venue in data:
    for paper in data[venue]:
        peer_reviews[venue][paper.id] = []
        for reply in paper.details["directReplies"]:
            if reply["invitation"].endswith("Official_Review"):
                peer_reviews[venue][paper.id].append(reply)


In [ ]:
with open("open_review_reviews_14_12_2023.json", "w") as outfile:
    json.dump(peer_reviews, outfile, indent=2)

In [ ]:
papers = defaultdict(dict)
json_data = defaultdict(dict)
for venue in data:
    if len(data[venue])==0:
        continue
    else:
        for paper in data[venue]:
            papers[venue][paper.id]=paper
            json_data[venue][paper.id]=paper

In [ ]:
with open("open_review_papers_plus_reviews_14_12_2023.json", "w") as outfile:
    json.dump(json_data, outfile, indent=2)

Double Blind submission which have peer reviews

In [ ]:
data2 = {}
for venue in venues:
    data2[venue] = client.get_all_notes(invitation=f"{venue}/-/Blind_Submission", details="directReplies")

In [ ]:
peer_reviews2 = defaultdict(dict)
for venue in data2:
    for paper in data2[venue]:
        peer_reviews2[venue][paper.id] = []
        for reply in paper.details["directReplies"]:
            if reply["invitation"].endswith("Official_Review"):
                peer_reviews2[venue][paper.id].append(reply)


In [ ]:
with open("open_review_reviews_14_12_2023_double_blind.json", "w") as outfile:
    json.dump(peer_reviews2, outfile, indent=2)

In [ ]:
papers2 = defaultdict(dict)
json_data2 = defaultdict(dict)
for venue in data2:
    if len(data2[venue])==0:
        continue
    else:
        for paper in data2[venue]:
            papers2[venue][paper.id]=paper
            json_data2[venue][paper.id]=paper.to_json()

In [ ]:
with open("open_review_papers_plus_reviews_14_12_2023_double_blind.json", "w") as outfile:
    json.dump(json_data2, outfile, indent=2)

In [ ]:
n_venues_sb=len([j for j in json_data if len(json_data[j])>0])
n_venues_db=len([j for j in json_data2 if len(json_data2[j])>0])
n_reviews_sb=0
n_papers_sb=0
n_reviews_db=0
n_papers_db=0
n_reviews_open_pa_sb=0
n_reviews_open_pa_db=0
non_anonymous_reviewers=defaultdict(int)
for v in peer_reviews.values():
    for p in v.values():
        if len(p)>0:
            n_papers_sb+=1
        for r in p:
            n_reviews_sb+=1
            identity=None
            for signature in r["signatures"]:
                if signature.startswith("~") or "@" in signature:
                    identity=signature
            if identity is not None:
                non_anonymous_reviewers[identity]+=1
                n_reviews_open_pa_sb+=1
                
for v in peer_reviews2.values():
    for p in v.values():
        if len(p)>0:
            n_papers_db+=1
        for r in p:
            n_reviews_db+=1
            identity=None
            for signature in r["signatures"]:
                if signature.startswith("~") or "@" in signature:
                    identity=signature
            if identity is not None:
                non_anonymous_reviewers[identity]+=1
                n_reviews_open_pa_db+=1


In [ ]:

print("Some stats:")
print(f"venues with single blind reviews: {n_venues_sb}")
print(f"venues with double blind reviews: {n_venues_db}")
print(f"total venues: {n_venues_db+n_venues_sb}")
print()
print(f"submissions single blind: {n_papers_sb}")
print(f"submissions double blind: {n_papers_db}")
print(f"total submissions: {n_papers_db+n_papers_sb}")
print()
print(f"reviews single blind: {n_reviews_sb}")
print(f"reviews double blind: {n_reviews_db}")
print(f"total reviews: {n_reviews_db+n_reviews_sb}")
print()
print(f"avg. reviews/submission single blind: {n_reviews_sb/n_papers_sb:.2f}")
print(f"avg. reviews/submission double blind: {n_reviews_db/n_papers_db:.2f}")
print(f"total avg. reviews/submission: {(n_reviews_db+n_reviews_sb)/(n_papers_db+n_papers_sb):.2f}")
print()
print(f"post acceptance deanonymized reviews single blind: {n_reviews_open_pa_sb}")
print(f"post acceptance deanonymized reviews double blind: {n_reviews_open_pa_db}")
print(f"total post acceptance deanonymized reviews: {n_reviews_open_pa_db+n_reviews_open_pa_sb}")


In [ ]:
non_anonymous_reviewer_profiles = openreview.tools.get_profiles(
    client,
    ids_or_emails=list(non_anonymous_reviewers.keys())
)


In [ ]:
for conf in json_data2:
    print(conf)

In [ ]:
for conf in json_data2:
    for paper in json_data2[conf]:
        dt_object = datetime.fromtimestamp(json_data2[conf][paper]["cdate"]/1000.0)
        print(dt_object.strftime("%Y-%m-%d %H:%M:%S"))
        print(json.dumps(json_data2[conf][paper], indent=3))
        break
    break

In [ ]:
peer_reviews.update(peer_reviews2)
papers.update(papers2)

In [ ]:
def get_year(conf_name):
    sub_names = conf_name.split("/")
    for name in sub_names:
        if len(name)==4 and name[:2] in ["19","20"] and name.isnumeric():
            return name
        
def get_conference(conf_name):
    conf_name = conf_name.split("/")[0]    
    return conf_name
    
def get_decision(paper):
    for reply in paper.details["directReplies"]:
        if reply["invitation"].endswith("Decision"):
            return reply["content"]["decision"]

def get_meta_review(paper):
    for reply in paper.details["directReplies"]:
        if reply["invitation"].endswith("Meta_Review"):
            return reply

def get_avg_score(reviews):
    return sum([r["score"] for r in reviews if r["score"] is not None])

def get_meta_score(reviews):
    return

def get_score(review, keys):
    if review is not None:
        score = None
        for kw in [
            "rating", 
            "final_rating", 
            "final_rating_after_the_rebuttal", 
            "overall_assessment",
            "overall_rating", 
            "review_rating", 
            "score",
            "Overall Score",
            "Overall score",
            "Q6 Overall score",
            "recommendation"]:
            if kw in review["content"]:
                # if kw=="Q6 Overall score":
                    # print(re.match("-?\d+",review["content"][kw]).group(0))
                try:
                    score = re.match("-?\d+",review["content"][kw])
                    if score is not None:
                        break
                except TypeError:
                    continue
        if score is not None:
            return int(score.group(0))
        else:
            keys|=keys|review["content"].keys()
review_keys=set()
confs_by_year = defaultdict(lambda: defaultdict(list))
for conf in peer_reviews:
    for paper in peer_reviews[conf]:
        decision = get_decision(papers[conf][paper])
        for pr in peer_reviews[conf][paper]:
            score = get_score(pr, review_keys)
            pr["score"] = score
        if decision is not None: 
            if len(peer_reviews[conf][paper])>0:
                confs_by_year[get_conference(conf)][get_year(conf)].append(
                    {
                        "decision": decision,
                        "avg_score": get_avg_score(peer_reviews[conf][paper]),
                        "reviews": peer_reviews[conf][paper]
                    }
                )
    

In [ ]:
review_keys

In [ ]:
for conf in confs_by_year:
    print(confs_by_year[conf].keys())

In [ ]:
print(json.dumps(non_anonymous_reviewer_profiles[0].to_json(), indent=3))

# API V2

In [ ]:
venue_group = client.get_group("ICLR.cc/2022/Conference")
print(venue_group.to_json().keys())

In [ ]:
print(venue_group.to_json()["readers"])

In [ ]:
json_data3 = {}
for venue_id in venues:
    try:
        venue_group = client.get_group(venue_id)
        submission_name = venue_group.content['submission_name']['value']
        submissions = client.get_all_notes(invitation=f'{venue_id}/-/{submission_name}', details='replies')
        for paper in submissions:
            json_data3[venue_id][paper.id]=paper.to_json()
        print(f"{venue_id} is API V2 venue.")
    except AttributeError:
        pass
    except openreview.OpenReviewException as e:
        print(e)

In [ ]:
json_data3

In [ ]:
venues_raw = client.get_group(id='venues').to_json()

In [ ]:
venues_raw.keys()